# Controller #

Import all neccessary packages

In [1]:
import os
import pandas as pd
import numpy as np

from controllers.inference_controllers.sandbox import Sandbox
from controllers.inference_controllers.generator import Generator
from controllers.inference_controllers.optimiser import Optimiser

from toolboxes.plotting_toolbox.domain import Domain
from toolboxes.inference_toolbox.parameter import Parameter
from toolboxes.inference_toolbox.model import Model
from toolboxes.inference_toolbox.likelihood import Likelihood

# current_directory = os.getcwd()
# if current_directory != '/project/':
#     os.chdir('/project/')

c:\Users\Sam\OneDrive\Documents\PhD\PhD_project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Defining simulated data

In [2]:
data_params = {
    'data_type': 'simulated_data',
    'model': {
        'model_select': 'log_gpm_norm',
        'model_params':{
            'H': 5
        },
        'inference_params':{
            'I_y': 1.6,
            'I_z': 1.2,
            'Q': 3e13,
        }
    },
    'domain':{
        'domain_select': 'cone_from_source_z_limited',
        'domain_params': {
            'r': 1000,
            'theta': np.pi/8,
            'source': [0,0,10]
        },
        'resolution': 20
    },
    'noise_dist': 'gaussian',
    'noise_level': 0.05,
    'output_header': 'Concentration',
}

prior_plots = [
    {
        'Q': [1e12, 5e13],
        'references':{
            'Q' : [2.82e13, 3.11e13, 2.89e13, 4.83e13],
            'labels': ['250m','200m','750m','1000m'] 
        }
    },
    {
        'I_y': [0.0001, 1.7],
        'I_z': [0.0001, 1.7],
        'references':{
            'I_y' : [0.22, 1.6, 0.11, 0.08, 0.06, 0.04],
            'I_z' : [0.2, 1.2, 0.08, 0.06, 0.03, 0.016],
            'labels': ['A','B','C','D','E','F'] 
        }
    }
]

results_name = 'simulated_data'

Defining default parameters and prior plots for gamma priors of the simulated data

In [3]:
default_params = {
    'infered_params':pd.Series({
        'model_params':pd.Series({
            'I_y_and_I_z': Parameter('I_y','I_z', prior_select='log_norm')
                                    .add_prior_param('mu', [0.4,0.4])
                                    .add_prior_param('cov', [[0.8,0],[0,0.8]]),
            'Q': Parameter('Q', prior_select = 'log_norm', order = 1e13).add_prior_param('mu',3).add_prior_param('cov',0.01),
        }),
        'likelihood_params':pd.Series({
            # 'sigma': Parameter('sigma', prior_select = 'log_norm').add_prior_param('mu', 0.05).add_prior_param('cov',0.05)
        })
    }),
    'model':Model('log_gpm_norm').add_model_param('H',5),
    'likelihood': Likelihood('gaussian_fixed_sigma').add_likelihood_param('sigma',0.05),
    'sampler': {
        'n_samples': 10000,
        'n_chains': 3,
        'thinning_rate': 1
    }
}

Running a general instance for gamma priors and simulated data

In [4]:
sandbox = Sandbox(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

visualiser = sandbox.run()
visualiser.get_summary()
visualiser.get_traceplot()
visualiser.get_autocorrelations()
visualiser.get_prior_plots(prior_plots)

domain = Domain('cone_from_source_z_limited', resolution=80)
domain.add_domain_param('r', 1000)
domain.add_domain_param('theta', np.pi/8)
domain.add_domain_param('source', [0,0,5])
domain.add_domain_param('z_slice', 0)
domain.add_domain_param('y_slice', 0)
domain.add_domain_param('x_slice', 100)

visualiser.visualise_results(domain = domain, plot_type = '2D_slice', name = 'small_scale', title='Log Concentration of Droplets', log_results=False)
visualiser.visualise_results(domain = domain, plot_type = '3D', name = 'small_scale', title='Log Concentration of Droplets', log_results=False)

Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Autocorrelations plot I_y, chain 1 already exists
Autocorrelations plot I_z, chain 1 already exists
Autocorrelations plot Q, chain 1 already exists
Autocorrelations plot I_y, chain 2 already exists
Autocorrelations plot I_z, chain 2 already exists
Autocorrelations plot Q, chain 2 already exists
Autocorrelations plot I_y, chain 3 already exists
Autocorrelations plot I_z, chain 3 already exists
Autocorrelations plot Q, chain 3 already exists
2D slice plot already exists!
2D slice plot already exists!


: 

In [ ]:
default_params = {
    'infered_params':pd.Series({
        'model_params':pd.Series({
            'I_y_and_I_z': Parameter('I_y','I_z', prior_select='log_norm')
                                    .add_prior_param('mu', [0.4,0.4])
                                    .add_prior_param('cov', [[0.8,0],[0,0.8]]),
            'Q': Parameter('Q', prior_select = 'log_norm', order = 1e13).add_prior_param('mu',3).add_prior_param('cov',0.01),
        }),
        'likelihood_params':pd.Series({
            # 'sigma': Parameter('sigma', prior_select = 'log_norm').add_prior_param('mu', 0.05).add_prior_param('cov',0.05)
        })
    }),
    'model':Model('log_gpm_norm').add_model_param('H',5),
    'likelihood': Likelihood('gaussian_fixed_sigma').add_likelihood_param('sigma',0.05),
    'sampler': {
        'n_samples': 10000,
        'n_chains': 1,
        'thinning_rate': 1
    }
}

Optimising the hyperparameters for gamma prior simulated data

In [ ]:
optimising_parameters = {
                    'I_y_and_I_z_mu_0': [1e-2, 2],
                    'I_y_and_I_z_mu_1': [1e-2, 2],
                    # 'I_y_and_I_z_cov_0_0': [1e-2, 2],
                    # 'I_y_and_I_z_cov_1_1': [1e-2, 2],
                }

optimiser = Optimiser(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

# AIC
study = optimiser.run(n_trials=100, optimiser_name='AIC_1', optimising_parameters=optimising_parameters, index_name='aic')
optimiser.get_plots(study)
domain = Domain('cone_from_source_z_limited', resolution=80)
domain.add_domain_param('r', 1000)
domain.add_domain_param('theta', np.pi/8)
domain.add_domain_param('source', [0,0,5])
domain.add_domain_param('z_slice', 0)
domain.add_domain_param('y_slice', 0)
domain.add_domain_param('x_slice', 100)

optimiser.run_best_params(study, domain, name = 'small_scale', prior_plots=prior_plots)

# # # # RMSE
# # # study = optimiser.run(n_trials=100, optimiser_name='RMSE_1', optimising_parameters=optimising_parameters, index_name='rmse')
# # # optimiser.get_plots(study)
# # # domain = Domain('cone_from_source_z_limited', resolution=80)
# # # domain.add_domain_param('r', 1000)
# # # domain.add_domain_param('theta', np.pi/8)
# # # domain.add_domain_param('source', [0,0,10])
# # # optimiser.run_best_params(study, domain, name = 'small_scale_3D_plots', prior_plots=prior_plots)

# # # # BIC
# # # study = optimiser.run(n_trials=100, optimiser_name='BIC_1', optimising_parameters=optimising_parameters, index_name='bic')
# # # optimiser.get_plots(study)
# # # domain = Domain('cone_from_source_z_limited', resolution=80)
# # # domain.add_domain_param('r', 1000)
# # # domain.add_domain_param('theta', np.pi/8)
# # # domain.add_domain_param('source', [0,0,10])
# # # optimiser.run_best_params(study, domain, name = 'small_scale_3D_plots', prior_plots=prior_plots)

[I 2024-03-04 00:47:55,568] A new study created in RDB with name: AIC_1
sample: 100%|██████████| 15000/15000 [00:05<00:00, 2968.75it/s, 13 steps of size 9.36e-02. acc. prob=0.94]
[I 2024-03-04 00:48:01,146] Trial 0 finished with value: -233.93898010253906 and parameters: {'I_y_and_I_z_mu_0': 1.1204688327762062, 'I_y_and_I_z_mu_1': 1.8083870967596618}. Best is trial 0 with value: -233.93898010253906.
sample: 100%|██████████| 15000/15000 [00:04<00:00, 3026.92it/s, 3 steps of size 1.10e-01. acc. prob=0.91] 
[I 2024-03-04 00:48:06,554] Trial 1 finished with value: -233.95272827148438 and parameters: {'I_y_and_I_z_mu_0': 1.9022440829403962, 'I_y_and_I_z_mu_1': 1.0127857323643896}. Best is trial 1 with value: -233.95272827148438.
sample: 100%|██████████| 15000/15000 [00:05<00:00, 2705.67it/s, 3 steps of size 1.11e-01. acc. prob=0.91] 
[I 2024-03-04 00:48:12,556] Trial 2 finished with value: -233.97299194335938 and parameters: {'I_y_and_I_z_mu_0': 1.324570513443766, 'I_y_and_I_z_mu_1': 0.3230